# ETL Project: Transform step

## 1. Import Libraries

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from datetime import datetime

## 2. Load Json Dataset

In [25]:
# Dataset extraction 

file_path = 'Data/Milan_Air_Quality.json'

df = pd.read_json(file_path)

if 'data' in df.columns:
    data = pd.json_normalize(df['data'])
    df.drop(columns='data', inplace=True)
    df = pd.concat([df, data], axis=1)

df.head()

,city_name,country_code,lat,lon,state_code,timezone,aqi,co,datetime,no2,o3,pm10,pm25,so2,timestamp_local,timestamp_utc,ts
0,Milan,IT,45.464,9.188,9,Europe/Rome,49,54.9,2024-09-06:15,2.0,107.0,13.0,9.00,3.0,2024-09-06T17:00:00,2024-09-06T15:00:00,1725634800
1,Milan,IT,45.464,9.188,9,Europe/Rome,49,55.1,2024-09-06:14,2.0,105.3,12.7,8.67,3.3,2024-09-06T16:00:00,2024-09-06T14:00:00,1725631200
2,Milan,IT,45.464,9.188,9,Europe/Rome,48,55.2,2024-09-06:13,2.0,103.7,12.3,8.33,3.7,2024-09-06T15:00:00,2024-09-06T13:00:00,1725627600
3,Milan,IT,45.464,9.188,9,Europe/Rome,47,55.4,2024-09-06:12,2.0,102.0,12.0,8.00,4.0,2024-09-06T14:00:00,2024-09-06T12:00:00,1725624000
4,Milan,IT,45.464,9.188,9,Europe/Rome,45,56.4,2024-09-06:11,2.3,97.7,12.0,8.00,4.3,2024-09-06T13:00:00,2024-09-06T11:00:00,1725620400


In [8]:
# Statistics and Info

df.info(), df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   city_name        72 non-null     object 
 1   country_code     72 non-null     object 
 2   lat              72 non-null     float64
 3   lon              72 non-null     float64
 4   state_code       72 non-null     int64  
 5   timezone         72 non-null     object 
 6   aqi              72 non-null     int64  
 7   co               72 non-null     float64
 8   datetime         72 non-null     object 
 9   no2              72 non-null     float64
 10  o3               72 non-null     float64
 11  pm10             72 non-null     float64
 12  pm25             72 non-null     float64
 13  so2              72 non-null     float64
 14  timestamp_local  72 non-null     object 
 15  timestamp_utc    72 non-null     object 
 16  ts               72 non-null     int64  
dtypes: float64(8), int

(None,
           lat     lon  state_code        aqi          co        no2  \
 count  72.000  72.000        72.0  72.000000   72.000000  72.000000   
 mean   45.464   9.188         9.0  62.527778   74.686111  10.486111   
 std     0.000   0.000         0.0  18.670586   15.124836   6.835972   
 min    45.464   9.188         9.0  15.000000   56.600000   1.000000   
 25%    45.464   9.188         9.0  55.750000   61.900000   4.000000   
 50%    45.464   9.188         9.0  64.000000   70.400000  10.150000   
 75%    45.464   9.188         9.0  75.000000   85.950000  15.300000   
 max    45.464   9.188         9.0  98.000000  106.200000  25.000000   
 
                o3       pm10       pm25        so2            ts  
 count   72.000000  72.000000  72.000000  72.000000  7.200000e+01  
 mean    71.563889  23.980556  16.467639   7.823611  1.724128e+09  
 std     42.250851   9.566471   6.585158   2.509054  7.534242e+04  
 min     17.000000   5.000000   3.000000   4.000000  1.724000e+09  
 25

## 2. Data Cleaning

In [3]:
# Check for missing values

print(f"DataFrame Shape before checking missing values: {df.shape}")

missing_values = df.isnull().sum()

if missing_values.any():
    print("\BE CAREFULL!! There are missing values in the following columns:")
    print(missing_values[missing_values > 0])
else:
    print("\nNo missing values in this DataFrame.")

DataFrame Shape before checking missing values: (72, 17)

No missing values in this DataFrame.


In [4]:
# Duplicates removal

initial_shape = df.shape
df = df.drop_duplicates()
final_shape = df.shape

if initial_shape != final_shape:
    print(f"\nDuplicated rows removed. New DataFrame shape: {final_shape}")
else:
    print("\nNo duplicates found in this DataFrame.")


No duplicates found in this DataFrame.


In [5]:

# Date/Hour column conversion to datetime format

df['timestamp_local'] = pd.to_datetime(df['timestamp_local'])
df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])

# Check datetime columns conversion
if pd.api.types.is_datetime64_any_dtype(df['timestamp_local']) and pd.api.types.is_datetime64_any_dtype(df['timestamp_utc']):
    print("\nThe columns 'timestamp_local' and 'timestamp_utc' have been successfully converted to datetime format.")
else:
    print("\nERROR: There was a problem converting the datetime columns.")
df.head()


The columns 'timestamp_local' and 'timestamp_utc' have been successfully converted to datetime format.


,city_name,country_code,lat,lon,state_code,timezone,aqi,co,datetime,no2,o3,pm10,pm25,so2,timestamp_local,timestamp_utc,ts
0,Milan,IT,45.464,9.188,9,Europe/Rome,84,70.4,2024-08-21:16,3.7,147.3,16.3,11.67,8.0,2024-08-21 18:00:00,2024-08-21 16:00:00,1724256000
1,Milan,IT,45.464,9.188,9,Europe/Rome,98,70.0,2024-08-21:15,2.0,158.0,14.0,10.00,8.0,2024-08-21 17:00:00,2024-08-21 15:00:00,1724252400
2,Milan,IT,45.464,9.188,9,Europe/Rome,95,69.8,2024-08-21:14,2.0,156.0,13.0,9.33,8.7,2024-08-21 16:00:00,2024-08-21 14:00:00,1724248800
3,Milan,IT,45.464,9.188,9,Europe/Rome,92,69.6,2024-08-21:13,2.0,154.0,12.0,8.67,9.3,2024-08-21 15:00:00,2024-08-21 13:00:00,1724245200
4,Milan,IT,45.464,9.188,9,Europe/Rome,90,69.4,2024-08-21:12,2.0,152.0,11.0,8.00,10.0,2024-08-21 14:00:00,2024-08-21 12:00:00,1724241600


In [6]:
# Drop feature with single value

for col in df.columns:

    if df[col].nunique() == 1:
        df.drop(columns=[col], inplace=True)

df.drop(columns='datetime', inplace=True)

df.head()

,aqi,co,no2,o3,pm10,pm25,so2,timestamp_local,timestamp_utc,ts
0,84,70.4,3.7,147.3,16.3,11.67,8.0,2024-08-21 18:00:00,2024-08-21 16:00:00,1724256000
1,98,70.0,2.0,158.0,14.0,10.00,8.0,2024-08-21 17:00:00,2024-08-21 15:00:00,1724252400
2,95,69.8,2.0,156.0,13.0,9.33,8.7,2024-08-21 16:00:00,2024-08-21 14:00:00,1724248800
3,92,69.6,2.0,154.0,12.0,8.67,9.3,2024-08-21 15:00:00,2024-08-21 13:00:00,1724245200
4,90,69.4,2.0,152.0,11.0,8.00,10.0,2024-08-21 14:00:00,2024-08-21 12:00:00,1724241600


## 3. Exploration Data Analysis

In [7]:
# 1. Histogram with Mean and Standard Deviation

# 'aqi', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2','timestamp_local', 'timestamp_utc', 'ts'

column = 'aqi'

mean_value = df[column].mean()
std_value = df[column].std()

fig_histogram = px.histogram(df, x=column, nbins=50, title=f'Distribution of {column.capitalize()} with Mean and STD')
fig_histogram.add_vline(x=mean_value, line_dash="dash", line_color="green", annotation_text="Mean")
fig_histogram.add_vline(x=mean_value + std_value, line_dash="dash", line_color="red", annotation_text="+1 STD")
fig_histogram.add_vline(x=mean_value - std_value, line_dash="dash", line_color="red", annotation_text="-1 STD")
fig_histogram.show()

In [11]:
# Box Plot

column = 'aqi'

fig_box = px.box(df, x=column, title=f'Box Plot of {column.capitalize()} by Category')
fig_box.show()

In [29]:
# Correlation Matrix Heatmap

correlation_matrix = df.corr()
fig_heatmap = px.imshow(round(correlation_matrix,2), title='Correlation Matrix Heatmap', color_continuous_scale='Magma',
                         aspect='auto', text_auto=True)
fig_heatmap.show()

In [54]:
# Time Series Plot

feature_to_viz = 'o3'

# you can use px.bar, px.line
fig_time_series = px.area(df, x='timestamp_local', y=feature_to_viz, title=f'Time Series of {feature_to_viz} with Rangeslider')
fig_time_series.update_xaxes(minor=dict(ticks="inside", showgrid=True))
fig_time_series.update_xaxes(rangeslider_visible=True)
fig_time_series.show()

In [25]:
#Scatter Plot with Regression Line

fig_scatter = px.scatter(df, x='pm10', y='pm25', title='Scatter Plot with Regression Line'
                         #,trendline='ols'
                         )
fig_scatter.show()

In [9]:
# Distribution plot with kde and histogram

df_numeric = df.select_dtypes(include='number')
features = df_numeric.columns

fig = go.Figure()
for feature in features:
    hist_data = [df[feature]]
    group_labels = [feature]
    distplot = ff.create_distplot(hist_data, group_labels, show_hist=True, show_rug=False)
    for trace in distplot.data:
        trace.visible = (feature == features[0]) 
        fig.add_trace(trace)

dropdown_buttons = [
    dict(
        label=feature,
        method='update',
        args=[{'visible': [i // 2 == j for i in range(len(features) * 2)]}]
    )
    for j, feature in enumerate(features)
]

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=dropdown_buttons,
            x=0.17,
            y=1.15,
            xanchor='right',
            yanchor='top'
        )
    ],
    title='Distribution and KDE of Selected Feature',
    xaxis_title='Feature Values',
    yaxis_title='Density',
    showlegend=True
)

fig.show()

## 4. Data Transformation

In [11]:
# Adding separated time columns: year, month, day, hour

df['year'] = df['timestamp_local'].dt.year
df['month'] = df['timestamp_local'].dt.month
df['day'] = df['timestamp_local'].dt.day
df['hour'] = df['timestamp_local'].dt.hour

df.head()

,aqi,co,no2,o3,pm10,pm25,so2,timestamp_local,timestamp_utc,ts,year,month,day,hour
0,84,70.4,3.7,147.3,16.3,11.67,8.0,2024-08-21 18:00:00,2024-08-21 16:00:00,1724256000,2024,8,21,18
1,98,70.0,2.0,158.0,14.0,10.00,8.0,2024-08-21 17:00:00,2024-08-21 15:00:00,1724252400,2024,8,21,17
2,95,69.8,2.0,156.0,13.0,9.33,8.7,2024-08-21 16:00:00,2024-08-21 14:00:00,1724248800,2024,8,21,16
3,92,69.6,2.0,154.0,12.0,8.67,9.3,2024-08-21 15:00:00,2024-08-21 13:00:00,1724245200,2024,8,21,15
4,90,69.4,2.0,152.0,11.0,8.00,10.0,2024-08-21 14:00:00,2024-08-21 12:00:00,1724241600,2024,8,21,14


In [12]:
# Create Time Index

df.set_index('timestamp_local', inplace=True)

df.head()

,aqi,co,no2,o3,pm10,pm25,so2,timestamp_utc,ts,year,month,day,hour
timestamp_local,,,,,,,,,,,,,
2024-08-21 18:00:00,84,70.4,3.7,147.3,16.3,11.67,8.0,2024-08-21 16:00:00,1724256000,2024,8,21,18
2024-08-21 17:00:00,98,70.0,2.0,158.0,14.0,10.00,8.0,2024-08-21 15:00:00,1724252400,2024,8,21,17
2024-08-21 16:00:00,95,69.8,2.0,156.0,13.0,9.33,8.7,2024-08-21 14:00:00,1724248800,2024,8,21,16
2024-08-21 15:00:00,92,69.6,2.0,154.0,12.0,8.67,9.3,2024-08-21 13:00:00,1724245200,2024,8,21,15
2024-08-21 14:00:00,90,69.4,2.0,152.0,11.0,8.00,10.0,2024-08-21 12:00:00,1724241600,2024,8,21,14


In [21]:
# Another possible transformation for this dataset could be adding a derived feature: Pollutant Ratio Features

df['pm10_pm25_ratio'] = round(df['pm10'] / df['pm25'], 2)
df['no2_o3_ratio'] = round(df['no2'] / df['o3'],2)
df['co_so2_ratio'] = round(df['co'] / df['so2'],2)

df = df[['aqi', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2', 'pm10_pm25_ratio', 'no2_o3_ratio', 'co_so2_ratio', 'timestamp_utc', 'ts',
       'year', 'month', 'day', 'hour']]

df.head()

,aqi,co,no2,o3,pm10,pm25,so2,pm10_pm25_ratio,no2_o3_ratio,co_so2_ratio,timestamp_utc,ts,year,month,day,hour
timestamp_local,,,,,,,,,,,,,,,,
2024-08-21 18:00:00,84,70.4,3.7,147.3,16.3,11.67,8.0,1.40,0.03,8.80,2024-08-21 16:00:00,1724256000,2024,8,21,18
2024-08-21 17:00:00,98,70.0,2.0,158.0,14.0,10.00,8.0,1.40,0.01,8.75,2024-08-21 15:00:00,1724252400,2024,8,21,17
2024-08-21 16:00:00,95,69.8,2.0,156.0,13.0,9.33,8.7,1.39,0.01,8.02,2024-08-21 14:00:00,1724248800,2024,8,21,16
2024-08-21 15:00:00,92,69.6,2.0,154.0,12.0,8.67,9.3,1.38,0.01,7.48,2024-08-21 13:00:00,1724245200,2024,8,21,15
2024-08-21 14:00:00,90,69.4,2.0,152.0,11.0,8.00,10.0,1.38,0.01,6.94,2024-08-21 12:00:00,1724241600,2024,8,21,14


## 5. Export Transformed Data

In [ ]:
trasformed_file_path = 'Data/Milan_Air_Quality_Transformed.json'

df.to_csv(trasformed_file_path)